In [1]:
import bcolz
import numpy as np
import pickle
import pandas as pd
import re
import os

In [2]:
GLOVE_PATH= "../../data/glove_6B"
FILEPATH = "./shoes_v2.csv"

In [3]:
words = []
idx = 0
word2idx = {}
vectors = bcolz.carray(np.zeros(1), rootdir=os.path.join(GLOVE_PATH,'glove.6B.50d.dat'), mode='w')

with open(os.path.join(GLOVE_PATH,'glove.6B.50d.txt'), 'rb') as f:
    for l in f:
        line = l.decode().split()
        word = line[0]
        words.append(word)
        word2idx[word] = idx
        idx += 1
        vect = np.array(line[1:]).astype(np.float)
        vectors.append(vect)
    
vectors = bcolz.carray(vectors[1:].reshape((400000, 50)), rootdir=os.path.join(GLOVE_PATH,'glove.6B.50d.dat'), mode='w')
vectors.flush()
pickle.dump(words, open(os.path.join(GLOVE_PATH, 'glove.6B.50d_words.pkl'), 'wb'))
pickle.dump(word2idx, open(os.path.join(GLOVE_PATH, 'glove.6B.50d_idx.pkl'), 'wb'))

In [4]:
vectors = bcolz.open(os.path.join(GLOVE_PATH, 'glove.6B.50d.dat'))[:]
words = pickle.load(open(os.path.join(GLOVE_PATH, 'glove.6B.50d_words.pkl'), 'rb'))
word2idx = pickle.load(open(os.path.join(GLOVE_PATH, 'glove.6B.50d_idx.pkl'), 'rb'))

glove = {w: vectors[word2idx[w]] for w in words}

In [6]:
descriptions = pd.read_csv('../../data/descriptions_df_v0.csv')
descriptions.head()

,Unnamed: 0,CID,Url,Descr0,Descr1,Descr2,Descr3,Descr4,Descr5,Descr6,Descr7,Descr8,Descr9
0,0,100627-255,Shoes/Oxfords/Bostonian/100627.255.jpg,Leather insole Bostonian Round Toe style black...,Oxfords Shoes Round Toe style for Men Lace up ...,Bostonian Lace up Oxfords Leather material Sho...,for Men Bostonian Oxfords Round Toe style bla...,Leather insole Bostonian Oxfords black for Me...,Round Toe style Bostonian Leather material Lea...,Leather material Oxfords Bostonian black Lace...,black Lace up Shoes Bostonian Round Toe style ...,Leather material Lace up Leather insole Oxford...,Leather insole Oxfords for Men Leather materi...
1,1,100627-72,Shoes/Oxfords/Bostonian/100627.72.jpg,for Men black Bostonian Lace up Leather insol...,Oxfords Capped Toe style black Shoes for Men ...,Capped Toe style Oxfords Lace up Bostonian fo...,black Bostonian Leather insole Oxfords Leather...,Bostonian Shoes Lace up for Men black Oxfords...,Shoes Capped Toe style Bostonian Oxfords Leat...,Bostonian Lace up Leather material Oxfords Lea...,Bostonian Oxfords Leather insole Shoes Capped ...,Bostonian Capped Toe style Oxfords Leather mat...,Leather insole Capped Toe style Oxfords for Me...
2,2,100657-216,Shoes/Oxfords/Bostonian/100657.216.jpg,Leather material Oxfords Round Toe style for M...,Shoes Oxfords Bostonian for Men Leather mater...,for Men black Shoes Leather material Oxfords B...,for Men Lace up Leather insole Bostonian blac...,Round Toe style black Bostonian Shoes Lace up...,for Men Leather material Round Toe style Shoes...,Leather material for Men Lace up Shoes black L...,Shoes black Round Toe style for Men Lace up B...,Lace up Oxfords Shoes Leather material for Me...,Bostonian Leather material Lace up Round Toe ...
3,3,100657-72,Shoes/Oxfords/Bostonian/100657.72.jpg,Leather material Capped Toe style for Men Bost...,Shoes Lace up Capped Toe style Leather insole ...,Oxfords Bostonian Lace up black Shoes for Men...,Capped Toe style for Men Oxfords black Bostoni...,Shoes for Men Leather insole Leather material...,Oxfords Bostonian Capped Toe style Leather ma...,Capped Toe style Shoes Leather insole Lace up ...,Capped Toe style Leather insole Oxfords Leathe...,Bostonian black for Men Capped Toe style Shoes...,Lace up Leather insole Leather material Cappe...
4,4,101026-3,Boots/Mid-Calf/Durango/101026.3.jpg,Oxfords Padded insole black white Lace up Sho...,for Men Leather material Bostonian Capped Toe ...,Oxfords black white Leather material Shoes for...,Shoes Lace up Capped Toe style Bostonian Leat...,Bostonian for Men Leather insole Oxfords Lace...,Oxfords Capped Toe style Shoes Lace up Leathe...,Capped Toe style Shoes Lace up Oxfords Bostoni...,Bostonian for Men Oxfords Shoes Padded insole ...,Lace up Oxfords black white for Men Capped Toe...,Leather material Capped Toe style Bostonian fo...


In [7]:
descr_cols = [f'Descr{d}' for d in range(10)]

descriptions[descr_cols] = descriptions[descr_cols].apply(lambda col: col.str.lower().str.split(), axis=0)

In [8]:
lens = descriptions[descr_cols].applymap(lambda desc: len(desc)).values
lens.min(), lens.max(), lens.mean()

(13, 27, 19.76049740311626)

In [9]:
max_len = 20
total_len = 1024

In [10]:
def embed(desc, max_len=max_len, total_len=total_len):
    emb = np.array([glove.get(w, glove['unk']) for w in desc[:max_len]]).reshape(-1)
    return np.pad(emb, (0, total_len - emb.shape[0]), mode='constant')

embeddings = descriptions[descr_cols].applymap(embed).values
embeddings.shape

(50060, 10)

In [11]:
embeddings_mat = np.zeros((embeddings.shape[0], embeddings.shape[1], total_len))
for i in range(embeddings.shape[0]):
    for j in range(embeddings.shape[1]):
        embeddings_mat[i, j, :] = embeddings[i, j]
embeddings_mat.shape

(50060, 10, 1024)

In [12]:
embeddings_mat[5425, 6]

array([-1.0878  ,  0.29802 ,  0.079287, ...,  0.      ,  0.      ,
        0.      ])

In [13]:
import pickle

pickle.dump(embeddings_mat, open("../../data/shoes_embeddings.pickle", "wb"))

In [ ]:
a = pickle.load(open('shoes_embeddings.pickle', 'rb'))
a = np.array(a)
a.shape